In [6]:
import os
import sys
import importlib
import json
import glob
import numpy as np
import tensorflow as tf

import util_recognition_network
import util_cochlear_model
import util_audio_transform

ARCH = 1
TASK = 'A'

fn_ckpt = 'models/recognition_networks/arch{}_task{}.ckpt*'.format(ARCH, TASK)
fn_ckpt = glob.glob(fn_ckpt)[-1].replace('.index', '')
fn_arch = 'models/recognition_networks/arch{}.json'.format(ARCH)
with open(fn_arch, 'r') as f_arch:
    list_layer_dict = json.load(f_arch)

fn_ckpt, fn_arch


('models/recognition_networks/arch1_taskA.ckpt-550000',
 'models/recognition_networks/arch1.json')

In [ ]:
importlib.reload(util_recognition_network)

tf.reset_default_graph()

if 'taskA' in fn_ckpt:
    n_classes_dict = {"/stimuli/labels_binary_via_int": 517}
else:
    n_classes_dict = {"/stimuli/word_int": 794}

var_scope = ''
with tf.variable_scope(var_scope):
    tensor_input = tf.placeholder(tf.float32, [None, 40, 20000, 1])
    tensor_output, tensors = util_recognition_network.build_network(
        tensor_input,
        list_layer_dict,
        n_classes_dict=n_classes_dict)

var_list = tf.get_collection(tf.GraphKeys.GLOBAL_VARIABLES, scope=var_scope)
var_dict = {v.name: v for v in var_list}

saver = tf.train.Saver(var_list=var_list, max_to_keep=0)

with tf.Session() as sess:
    sess.run(tf.global_variables_initializer())
    var_dict_init = sess.run(var_dict)
    saver.restore(sess, fn_ckpt)
    var_dict_load = sess.run(var_dict)
    
#     save_path = fn_ckpt.replace('trash/', '')
#     print('WRITING: {}'.format(save_path))
#     saver_to_save.save(
#         sess,
#         save_path=save_path,
#         global_step=None,
#         latest_filename=None,
#         meta_graph_suffix='meta',
#         write_meta_graph=False,
#         write_state=False,
#         strip_default_attrs=False)


In [ ]:
for v in var_list:
    print(v.name, np.mean(var_dict_init[v.name]), np.mean(var_dict_load[v.name]))


In [5]:
import util_cochlear_model
importlib.reload(util_cochlear_model)
tf.reset_default_graph()

tensor_waveform = tf.placeholder(tf.float32, [None, 40000])

util_cochlear_model.build_cochlear_model(
    tensor_waveform,
    signal_rate=20000,
    filter_type='half-cosine',
    filter_spacing='erb',
    HIGH_LIM=8000,
    LOW_LIM=20,
    N=40,
    SAMPLE_FACTOR=1,
    bandwidth_scale_factor=1.0,
    compression='stable_point3',
    include_highpass=False,
    include_lowpass=False,
    linear_max=1.0,
    rFFT=True,
    rectify_and_lowpass_subbands=True,
    return_subbands_only=True)


[make_cos_filters_nx] using filter_spacing=`erb`


(<tf.Tensor 'ExpandDims_1:0' shape=(?, 40, 20000, 1) dtype=float32>,
 {'fft_input': <tf.Tensor 'exd_fft_of_input:0' shape=(?, 1, 20001) dtype=complex64>,
  'filts_tensor': <tf.Tensor 'ExpandDims:0' shape=(1, 40, 20001) dtype=complex64>,
  'input_real': <tf.Tensor 'Placeholder:0' shape=(?, 40000) dtype=float32>,
  'input_signal': <tf.Tensor 'Placeholder:0' shape=(?, 40000) dtype=float32>,
  'output_tfcoch_graph': <tf.Tensor 'ExpandDims_1:0' shape=(?, 40, 20000, 1) dtype=float32>,
  'subbands': <tf.Tensor 'mul_subbands:0' shape=(?, 40, 20001) dtype=complex64>,
  'subbands_ifft': <tf.Tensor 'ifft_subbands:0' shape=(?, 40, 40000) dtype=float32>,
  'subbands_time': <tf.Tensor 'ifft_subbands:0' shape=(?, 40, 40000) dtype=float32>,
  'subbands_time_lowpassed': <tf.Tensor 'transpose_2:0' shape=(?, 40, 20000) dtype=float32>,
  'subbands_time_lowpassed_compressed': <tf.Tensor 'ExpandDims_1:0' shape=(?, 40, 20000, 1) dtype=float32>,
  'subbands_time_processed': <tf.Tensor 'ExpandDims_1:0' shape=(

In [8]:
import util_audio_transform
importlib.reload(util_audio_transform)

tf.reset_default_graph()

tensor_waveform = tf.placeholder(tf.float32, [None, 40000])

UNET_PARAMS = {
    "num_initial_filters": 24,
    "output_activation": "identity"
}

var_scope = 'frontend_model'
with tf.variable_scope(var_scope):
    tensor_waveform_unet = util_audio_transform.build_unet(
        tensor_waveform,
        signal_rate=20000,
        UNET_PARAMS=UNET_PARAMS)

var_list = tf.get_collection(tf.GraphKeys.GLOBAL_VARIABLES, scope=var_scope)
var_dict = {v.name: v for v in var_list}

saver = tf.train.Saver(var_list=var_list, max_to_keep=0)

fn_ckpt = 'models/audio_transforms/unet_A123/frontend_model.ckpt-600000'
with tf.Session() as sess:
    sess.run(tf.global_variables_initializer())
    var_dict_init = sess.run(var_dict)
    saver.restore(sess, fn_ckpt)
    var_dict_load = sess.run(var_dict)


Instructions for updating:
Use standard file APIs to check for files with this prefix.
INFO:tensorflow:Restoring parameters from models/audio_transforms/unet_A123/frontend_model.ckpt-600000


In [9]:
for k in var_dict_init.keys():
    print(k, np.mean(var_dict_init[k]), np.mean(var_dict_load[k]))


frontend_model/separator/conv1d_8/kernel:0 -9.457302e-06 -0.0019786328
frontend_model/separator/conv1d_12/kernel:0 1.4399326e-05 -0.00044223526
frontend_model/separator/conv1d_9/kernel:0 7.0753454e-06 -0.0017836875
frontend_model/separator/conv1d_3/bias:0 0.0 0.003133187
frontend_model/separator/conv1d_6/bias:0 0.0 -0.008983766
frontend_model/separator/interp_10:0 0.015453177 0.0122272335
frontend_model/separator/conv1d_5/kernel:0 -3.7155063e-05 -0.0024077457
frontend_model/separator/conv1d_11/bias:0 0.0 -0.021831416
frontend_model/separator/conv1d_19/bias:0 0.0 -0.029271709
frontend_model/separator/conv1d_2/bias:0 0.0 0.0035906392
frontend_model/separator/interp_11:0 -0.0010926673 -0.012157106
frontend_model/separator/conv1d_20/bias:0 0.0 -0.018392818
frontend_model/separator/conv1d_14/kernel:0 -4.1312596e-05 -0.00059503503
frontend_model/separator/conv1d_22/kernel:0 -7.215842e-05 -0.00079732825
frontend_model/separator/interp_4:0 -0.004335786 -0.005335737
frontend_model/separator/con

In [15]:
import os
import sys
import glob
import json
import numpy as np
import tensorflow as tf
import scipy.io.wavfile
import IPython.display as ipd

import util_audio_transform


regex_fn_wav = 'audio/ex*_unprocessed_input.wav'
list_fn_wav = glob.glob(regex_fn_wav)


def get_rms(y):
    return np.sqrt(np.mean(np.square(y - np.mean(y))))

def set_rms(y, rms):
    y = y - np.mean(y)
    return rms * y / get_rms(y)

list_y = []
for fn_wav in list_fn_wav:
    sr, y = scipy.io.wavfile.read(fn_wav)
    y = set_rms(y, 0.02)
    assert sr == 20e3
    list_y.append(y)
list_y = np.array(list_y)
list_y.shape


(10, 40000)

In [16]:
tf.reset_default_graph()

fn_ckpt = 'models/audio_transforms/unet_A123/frontend_model.ckpt-600000'

tensor_waveform = tf.placeholder(tf.float32, [None, 40000])

var_scope = 'frontend_model'
with tf.variable_scope(var_scope):
    tensor_waveform_unet = util_audio_transform.build_unet(
        tensor_waveform,
        signal_rate=20000,
        UNET_PARAMS={})

var_list = tf.get_collection(tf.GraphKeys.GLOBAL_VARIABLES, scope=var_scope)
saver = tf.train.Saver(var_list=var_list, max_to_keep=0)
with tf.Session() as sess:
    sess.run(tf.global_variables_initializer())
    saver.restore(sess, fn_ckpt)
    list_y_unet = sess.run(tensor_waveform_unet, feed_dict={tensor_waveform: list_y})


INFO:tensorflow:Restoring parameters from models/audio_transforms/unet_A123/frontend_model.ckpt-600000


In [17]:
for itr0 in range(list_y_unet.shape[0]):
    y = list_y[itr0]
    y_unet = list_y_unet[itr0]
    print(itr0)
    ipd.display(ipd.Audio(y, rate=sr))
    ipd.display(ipd.Audio(y_unet, rate=sr))


0


1


2


3


4


5


6


7


8


9


In [10]:
np.sqrt(np.mean(np.square(y)))


0.18189876

In [12]:
np.mean(y)

-0.0007864829